In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/sales_transactions.csv')
df.head()

,Transaction_ID,Customer_ID,Region,Product_Category,Product_Name,Quantity,Unit_Price ($),Total_Sales ($),Transaction_Date,Payment_Method
0,10001,2861,East,Books,Biography,3,155.58,466.74,2024-09-09,PayPal
1,10002,3295,West,Toys,Action Figure,8,439.08,3512.64,2023-09-14,PayPal
2,10003,3131,North,Toys,Doll,8,276.15,2209.20,2024-08-17,Credit Card
3,10004,3096,North,Electronics,Laptop,2,291.73,583.46,2025-01-04,PayPal
4,10005,3639,South,Beauty,Perfume,7,332.55,2327.85,2024-03-09,PayPal


In [ ]:
# checking for null values
df.isna().sum() #no missing values
df.duplicated().sum() # No duplicate value

# standardizing column names
# first convert all to lower case, then replace $ with _USD and replace every
# with underscore
df.columns = df.columns.str.lower().str.replace(' ($)', '_(usd)').str.replace(' ', '_')

# checking the basic info of the df
df.describe()

# converting the date column to proper datetime type
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   transaction_id     10000 non-null  int64         
 1   customer_id        10000 non-null  int64         
 2   region             10000 non-null  object        
 3   product_category   10000 non-null  object        
 4   product_name       10000 non-null  object        
 5   quantity           10000 non-null  int64         
 6   unit_price_(usd)   10000 non-null  float64       
 7   total_sales_(usd)  10000 non-null  float64       
 8   transaction_date   10000 non-null  datetime64[ns]
 9   payment_method     10000 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 781.4+ KB


# 1️⃣ Basic Summary Statistics

In [ ]:
# Calculate the total revenue across all transactions.
total_revenue = df['total_sales_(usd)'].sum()
print(f"The total revenue obtained from all transactions was ${total_revenue:.0f}")

# Find the average transaction value (Total_Sales).
avg_txn_value = df['total_sales_(usd)'].mean()
print(f"The average transaction value was ${avg_txn_value:.2f}")

# Get the number of unique customers.
unique_customers = df['customer_id'].nunique()
print(f"There were {unique_customers} unique customers")

# Find the highest and lowest Total_Sales values.
max_total_sales = df['total_sales_(usd)'].max()
min_total_sales = df['total_sales_(usd)'].min()
print(f"The maximum total sales was ${max_total_sales:.2f} and the minimum total sales was ${min_total_sales:.2f}")

# Compute the average number of products purchased per transaction.
avg_quantity = df['quantity'].mean()
print(f"The average quantity of items purchased per transaction was {avg_quantity:.2f}")

The total revenue obtained from all transactions was $13997042
The average transaction value was $1399.70
There were 2889 unique customers
The maximum total sales was $4990.90 and the minimum total sales was $5.33
The average quantity of items purchased per transaction was 5.53


# 2️⃣ Aggregation Using `.groupby()`

In [ ]:
# Find total revenue per Region.
revenue_by_region = df.groupby('region')['total_sales_(usd)'].agg('sum').reset_index()
#sorting the values in descending order
revenue_by_region.sort_values(
    by='total_sales_(usd)', ascending=False, inplace=True
    )
#reseting the column names of the new table
revenue_by_region.columns = ['Region', 'Total Revenue']
revenue_by_region


# Compute average Total_Sales per Product_Category.
avg_sales_per_category = df.groupby('product_category')['total_sales_(usd)'].agg('mean').reset_index()
avg_sales_per_category.columns = ['Product Category', 'Average Sales'] #renaming columns
avg_sales_per_category

txns_per_payment_method = df.groupby('payment_method')['transaction_id'].agg('count').reset_index()
txns_per_payment_method.columns = ['Payment Method', 'Number of Transactions']
# reorder the table
txns_per_payment_method.sort_values(by = 'Number of Transactions', inplace = True)
txns_per_payment_method

prod_sales_by_reg = df.groupby(['region', 'product_name'])['total_sales_(usd)'].agg('sum').reset_index()
max_sales_by_reg = prod_sales_by_reg.loc[prod_sales_by_reg.groupby('region')['total_sales_(usd)'].idxmax()]
max_sales_by_reg

,region,product_name,total_sales_(usd)
22,East,Tablet,162335.31
25,North,Biography,189718.57
56,South,Foundation,177236.21
92,West,Sneakers,221582.62


In [ ]:
for index, row in max_sales_by_reg.iterrows():
  print(f"{row['product_name']} is the highest selling product in the {row['region']}")

Tablet is the highest selling product in the East
Biography is the highest selling product in the North
Foundation is the highest selling product in the South
Sneakers is the highest selling product in the West


In [ ]:
total_customer_sales = df.groupby('customer_id')['total_sales_(usd)'].sum().reset_index()
total_customer_sales.columns = ['Customer ID', 'Total Sales']

,Customer ID,Total Sales
0,2001,255.86
1,2002,8164.15
2,2003,1260.19
3,2004,3755.70
4,2005,372.10
...,...,...
2884,4995,4672.10
2885,4996,6069.52
2886,4997,2459.72
2887,4998,88.55


# 3️⃣ Multi-Level Aggregation

In [ ]:
# Calculate the total revenue for each product category in each region
# Group by 'region' and 'product_category', then sum the 'total_sales_(usd)' column
total_revenue_by_category_region = df.groupby(['region', 'product_category'])['total_sales_(usd)'].agg('sum').reset_index()
total_revenue_by_category_region

# Calculate the average sales and average quantity for each product
# Group by 'product_name', then calculate the mean of 'total_sales_(usd)' and 'quantity'
avg_sales_and_quantity_per_product = df.groupby('product_name').agg({
    'total_sales_(usd)': 'mean',  # Average sales per product
    'quantity': 'mean'             # Average quantity per product
}).reset_index()

# Rename the columns for clarity
avg_sales_and_quantity_per_product.columns = ['Product Name', 'Average Sales', 'Average Quantity']
avg_sales_and_quantity_per_product

# Find the top 3 best-selling products in each region based on total sales
# Group by 'region' and 'product_name', then sum the 'total_sales_(usd)' column
top_3_products_by_region = df.groupby(['region', 'product_name'])['total_sales_(usd)'].sum().reset_index()

# For each region, get the top 3 products with the highest total sales
top_3_products_by_region = top_3_products_by_region.groupby('region').apply(
    lambda x: x.nlargest(3, 'total_sales_(usd)')  # Get the top 3 rows based on 'total_sales_(usd)'
).reset_index(drop=True)  # Reset index to remove multi-indexing
top_3_products_by_region

# Calculate the average price per product for each payment method
# Group by 'payment_method', then calculate the mean of 'unit_price_(usd)'
avg_price_per_product_method = df.groupby('payment_method')['unit_price_(usd)'].mean().reset_index()

# Rename the columns for clarity
avg_price_per_product_method.columns = ['Payment Method', 'Average Price']
avg_price_per_product_method

# Find the most purchased product (by quantity) in each region
# Group by 'region' and 'product_name', then sum the 'quantity' column
most_purchased_per_region = df.groupby(['region', 'product_name'])['quantity'].sum().reset_index()

# For each region, get the product with the highest quantity purchased
most_purchased_per_region = most_purchased_per_region.groupby('region').apply(
    lambda x: x.nlargest(1, 'quantity')  # Get the row with the highest 'quantity'
).reset_index(drop=True)  # Reset index to remove multi-indexing
most_purchased_per_region

<ipython-input-44-d41309ab3c15>:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_3_products_by_region = top_3_products_by_region.groupby('region').apply(
<ipython-input-44-d41309ab3c15>:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  most_purchased_per_region = most_purchased_per_region.groupby('region').apply(


,region,product_name,quantity
0,East,Tablet,647
1,North,Biography,715
2,South,Foundation,712
3,West,Sneakers,811


In [ ]:
for _, row in most_purchased_per_region.iterrows():
  print(f"{row['product_name']} is the most purchased product in the {row['region']}")

Tablet is the most purchased product in the East
Biography is the most purchased product in the North
Foundation is the most purchased product in the South
Sneakers is the most purchased product in the West


# 4️⃣ Pivot Tables for Aggregation

In [59]:
sales_by_region_and_category = df.pivot_table(
    values = 'total_sales_(usd)', # COLUMN TO AGGREGATE
    index = 'region',     #column to act as index
    columns = 'product_category',
    aggfunc = 'sum', # aggregation functon
    fill_value = 0 # handling NaN
)

quantity_sold_per_payment_method = df.pivot_table(
    values='quantity',          # Column to aggregate (quantity sold)
    index='payment_method',     # Group by payment method
    aggfunc='sum',              # Sum the quantities
    fill_value=0                # Replace missing values with 0
)

# Extract the month name from the transaction date and create a new column
df['txn_month'] = df['transaction_date'].dt.month_name()

average_sales_per_month = df.pivot_table(
    values='total_sales_(usd)',  # Column to aggregate (total sales in USD)
    index='txn_month',           # Group by month
    aggfunc='mean',              # Calculate the average sales
    fill_value=0                 # Replace missing values with 0
)

total_sales_by_region_and_payment_method = df.pivot_table(
    values='total_sales_(usd)',  # Column to aggregate (total sales in USD)
    index=['region', 'payment_method'],  # Group by region and payment method
    aggfunc='sum',               # Sum the total sales
    fill_value=0                 # Replace missing values with 0
)


product_category_by_revenue = df.pivot_table(
    values='total_sales_(usd)',  # Column to aggregate (total sales in USD)
    index='product_category',    # Group by product category
    aggfunc='sum',               # Sum the total sales
    fill_value=0                 # Replace missing values with 0
)

product_category_by_revenue.sort_values(
    by='total_sales_(usd)',      # Column to sort by (total sales in USD)
    ascending=False,             # Sort in descending order (highest to lowest)
    inplace=True                 # Modify the DataFrame in place
)

product_category_by_revenue

,total_sales_(usd)
product_category,
Clothing,2495466.98
Electronics,2378550.59
Toys,2316393.47
Home & Kitchen,2310569.78
Beauty,2269476.72
Books,2226584.53


# 5️⃣ Rolling & Cumulative Aggregations

In [63]:
# Compute a cumulative sum of Total_Sales over time.
df['cumulative_total_sales'] = df['total_sales_(usd)'].cumsum()
df.head()

# Calculate a rolling 7-day average of Total_Sales.
df['rolling_7_day_avg'] = df['total_sales_(usd)'].rolling(window=7).mean()
df.head()

# Find the monthly revenue trend using a time-based rolling sum
df['transaction_month'] = df['transaction_date'].dt.to_period('M')
monthly_revenue = df.groupby('transaction_month')['total_sales_(usd)'].sum().reset_index()
monthly_revenue

# Compute a year-over-year sales growth percentage.
df['year'] = df['transaction_date'].dt.year
year_over_year_growth = df.groupby('year')['total_sales_(usd)'].sum().pct_change() * 100
year_over_year_growth


,total_sales_(usd)
year,
2023,NaN
2024,23.058132
2025,-78.642108
